In [11]:
%load_ext autoreload
%autoreload 2

from webbrowser import get
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *
import logging
from pretty_logger import get_logger
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# define subset (ignore all other data)
ema = [f"Y{i}" for i in range(1, 8, 1)]
physical = [f"P{i}" for i in range(1, 5, 1)]
social = [f"S{i}" for i in range(1, 8, 1)]

In [13]:
df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df["C"] = df["date"].apply(
    lambda date: (
        COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID
    )
)

df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)

subset = ema + physical + social + sleep + ["C"]
df = df[subset]
df.dropna(inplace=True)

sets_df = pd.read_parquet(sets_file, engine="pyarrow")

In [14]:
skip_emas = list(set(ema_dictionary.keys()).difference(set(["Y2", "Y3"])))
skip_treatments = list(
    set(subset).difference(
        set(
            [
                "P1",
                "P4",
                "S1",
                "S2",
                "S3",
                "S4",
                "S5",
                "S6",
                "Z1",
                "Z2",
                "Z3",
            ]
        )
    )
)

skip_treatments

['Y5', 'P3', 'Y3', 'S7', 'Y4', 'C', 'Y7', 'Y2', 'P2', 'Y6', 'Y1']

In [15]:
# index = 17
# model_row = sets_df.iloc[index]

ccp_alphas = [0]
n_estimatorss = [100, 400, 1200]
n_estimators = n_estimatorss[0]
for n_estimators in n_estimatorss:
    for ccp_alpha in ccp_alphas:
        for index, model_row in sets_df.iterrows():
            already_fitted_sets = []

            covariate_set = CovariateSet(
                row=model_row,
                data=df,
                outcomes_to_skip=skip_emas,
                treatments_to_skip=skip_treatments,
            )

            if not covariate_set.valid_set:
                logger.debug(f"Skipping {covariate_set} (no valid set)")
                continue

            if covariate_set.set_to_fit in already_fitted_sets:
                logger.info(
                    f"Skipping {covariate_set} (already did {covariate_set.set_to_fit})"
                )
                continue

            already_fitted_sets.append(covariate_set.set_to_fit)
            logger.info(f"Fitting\n{covariate_set!r}")
            logger.info(
                f"Median of {covariate_set.treatment} {df[covariate_set.treatment].median()}"
            )
            wbm = WBRandomForestModel(
                data=df,
                n_estimators=n_estimators,
                ccp_alpha=ccp_alpha,
                treatment=covariate_set.treatment,
                outcome=covariate_set.outcome,
                separating_set=covariate_set.restricted_adjustment_set,
                name=f"row:{index}",
            )
            logger.info(
                f"\nccp_alpha ={ccp_alpha}, n_estimators={n_estimators}, "
                f"pre_rsq train={wbm.pre_r_squared[0]}, pre_rsq test={wbm.pre_r_squared[1]}\n"
                f"pre mae test ={wbm.pre_r_squared[2]}\n"
                f"post_rsq train={wbm.post_r_squared[0]}, post_rsq test={wbm.post_r_squared[1]}\n"
            )
            logger.info(
                "-----------------------------------------------------\n"
            )

[2024-05-28 14:02:53.364] - modelslog- INFO N/A- 1203852471.py: 29 [<module>]: Fitting
treatment: P1:excercise (seconds), outcome: Y2:phq4_score, adjustment set={'S2', 'S4', 'S1', 'P4', 'S6', 'P2', 'S3'}
[2024-05-28 14:02:53.365] - modelslog- INFO N/A- 1203852471.py: 30 [<module>]: Median of P1 10963.4


[2024-05-28 14:02:56.473] - modelslog- INFO N/A- 1203852471.py: 42 [<module>]: 
ccp_alpha =0, n_estimators=100, pre_rsq train=0.8796439484711864, pre_rsq test=0.1481680666564129
pre mae test =1.7575477272590125
post_rsq train=0.8807983195552385, post_rsq test=0.2227415385556828

[2024-05-28 14:02:56.473] - modelslog- INFO N/A- 1203852471.py: 48 [<module>]: -----------------------------------------------------

[2024-05-28 14:02:56.474] - modelslog- INFO N/A- 1203852471.py: 29 [<module>]: Fitting
treatment: P4:sports (hours), outcome: Y2:phq4_score, adjustment set={'S5', 'P3', 'S4', 'S1', 'P2', 'S3'}
[2024-05-28 14:02:56.475] - modelslog- INFO N/A- 1203852471.py: 30 [<module>]: Median of P4 0.08325
[2024-05-28 14:02:59.565] - modelslog- INFO N/A- 1203852471.py: 42 [<module>]: 
ccp_alpha =0, n_estimators=100, pre_rsq train=0.8659126053242656, pre_rsq test=0.10325531340163985
pre mae test =1.830383262678301
post_rsq train=0.8820738684377533, post_rsq test=0.14117039576805768

[2024-05-28 